In [145]:
 # Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import time

# Import API key
from api_keys import g_key

#Access maps with unique API key
gmaps.configure(api_key=g_key)

In [146]:
city_weather_df = pd.read_csv("../resources/cities.csv")
city_weather_df

,City,Lat,Lng,Max Temp,Humidity,Clouds,Wind Speed,Country,Date
0,Jamestown,42.0970,-79.2353,42.39,76,98,7.00,US,1638496676
1,Busselton,-33.6500,115.3333,70.09,33,0,12.91,AU,1638496955
2,Mataura,-46.1927,168.8643,67.82,72,97,3.49,NZ,1638496709
3,Upernavik,72.7868,-56.1549,21.16,88,100,25.68,GL,1638496701
4,Rikitea,-23.1203,-134.9692,76.28,75,7,15.30,PF,1638496956
...,...,...,...,...,...,...,...,...,...
547,Illora,37.2877,-3.8811,38.10,89,0,2.30,ES,1638497116
548,Lajas,18.0500,-67.0593,78.39,83,8,1.01,PR,1638497116
549,Booué,-0.0921,11.9385,72.99,99,100,1.14,GA,1638497116
550,Flinders,-34.5833,150.8552,83.80,74,23,18.81,AU,1638497117


In [147]:
# Humidity Heat map

In [148]:
#configure gmaps
gmaps.configure(api_key=g_key)

#Use the Lat and Lng as locations
locations = city_weather_df[["Lat","Lng"]]

#Add Heatmap layer to map
humidity = city_weather_df["Humidity"]

In [149]:
#Plot heatmap
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)
#Add heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=np.max(humidity), point_radius=3)

fig.add_layer(heat_layer)
fig 

Figure(layout=FigureLayout(height='420px'))

In [150]:
#Narrow down the cities to fit weather conditions
#Drop any rows will null values
smaller_city_df = city_weather_df.loc[(city_weather_df["Wind Speed"] <= 10) & (city_weather_df["Clouds"]== 0) & \
                  (city_weather_df["Max Temp"] >= 70) & (city_weather_df["Max Temp"] <= 80)].dropna()
smaller_city_df

,City,Lat,Lng,Max Temp,Humidity,Clouds,Wind Speed,Country,Date
186,Berbera,10.4396,45.0143,74.71,61,0,3.58,SO,1638497011
196,Mossamedes,-15.1961,12.1522,71.17,80,0,4.70,AO,1638497014
247,Quelimane,-17.8786,36.8883,76.77,84,0,5.73,MZ,1638497029
261,Ambilobe,-13.2000,49.0500,72.05,75,0,3.49,MG,1638497033
267,Capitán Bado,-23.2667,-55.5333,70.39,53,0,5.59,PY,1638497035
289,Arroyos y Esteros,-25.0667,-57.1000,76.75,61,0,5.37,PY,1638497041
321,Beloha,-25.1667,45.0500,70.95,79,0,9.89,MG,1638496810
343,Naviraí,-23.0650,-54.1906,71.92,48,0,2.48,BR,1638496820
387,Salalah,17.0151,54.0924,73.49,56,0,2.30,OM,1638497070
474,Cuamba,-14.8031,36.5372,72.99,65,0,6.33,MZ,1638497095


In [151]:
# Hotel Map

In [152]:
 #Store into variable named hotel_df
hotel_df = smaller_city_df.loc[:,["City","Country","Lat","Lng"]]

#Add a "Hotel Name" column to the DataFrame
hotel_df["Hotel Name"] = ""
hotel_df 

,City,Country,Lat,Lng,Hotel Name
186,Berbera,SO,10.4396,45.0143,
196,Mossamedes,AO,-15.1961,12.1522,
247,Quelimane,MZ,-17.8786,36.8883,
261,Ambilobe,MG,-13.2000,49.0500,
267,Capitán Bado,PY,-23.2667,-55.5333,
289,Arroyos y Esteros,PY,-25.0667,-57.1000,
321,Beloha,MG,-25.1667,45.0500,
343,Naviraí,BR,-23.0650,-54.1906,
387,Salalah,OM,17.0151,54.0924,
474,Cuamba,MZ,-14.8031,36.5372,


In [153]:
 #Set parameters to search for hotels with 5000 meters.
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [154]:
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    params["location"] = f"{lat},{lng}"
    
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    results = response['results']
    
    #Save the hotel name to the dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
    except (KeyError, IndexError):
        print("Could not find result")
        print("-------------------------")
        #delay the code by one second to avoid an error
        time.sleep(1)
    print("--------------------Ending Search")

Retrieving Results for Index 186: Berbera.
Could not find result
-------------------------
--------------------Ending Search
Retrieving Results for Index 196: Mossamedes.
Could not find result
-------------------------
--------------------Ending Search
Retrieving Results for Index 247: Quelimane.
Could not find result
-------------------------
--------------------Ending Search
Retrieving Results for Index 261: Ambilobe.
Could not find result
-------------------------
--------------------Ending Search
Retrieving Results for Index 267: Capitán Bado.
Could not find result
-------------------------
--------------------Ending Search
Retrieving Results for Index 289: Arroyos y Esteros.
Could not find result
-------------------------
--------------------Ending Search
Retrieving Results for Index 321: Beloha.
Could not find result
-------------------------
--------------------Ending Search
Retrieving Results for Index 343: Naviraí.
Could not find result
-------------------------
--------------

In [155]:
hotel_df


,City,Country,Lat,Lng,Hotel Name
186,Berbera,SO,10.4396,45.0143,
196,Mossamedes,AO,-15.1961,12.1522,
247,Quelimane,MZ,-17.8786,36.8883,
261,Ambilobe,MG,-13.2000,49.0500,
267,Capitán Bado,PY,-23.2667,-55.5333,
289,Arroyos y Esteros,PY,-25.0667,-57.1000,
321,Beloha,MG,-25.1667,45.0500,
343,Naviraí,BR,-23.0650,-54.1906,
387,Salalah,OM,17.0151,54.0924,
474,Cuamba,MZ,-14.8031,36.5372,


In [156]:
 # NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]] 

In [157]:
 # Plot markers on top of the heatmap
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))